In [1]:
#import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

import researchpy as rp
from scipy.stats import chi2_contingency
from statsmodels.sandbox.stats.multicomp import multipletests # for multiple comparisons correction
from itertools import combinations  # for post-hoc tests

In [2]:
# assists in displaying significance
# using the work from https://neuhofmo.github.io/chi-square-and-post-hoc-in-python/
def get_asterisks_for_pval(p_val, alpha=0.05):
    """Receives the p-value and returns asterisks string."""
    if p_val > alpha:  # bigger than alpha
        p_text = "ns"
    # following the standards in biological publications
    elif p_val < 1e-4:  
        p_text = '****'
    elif p_val < 1e-3:
        p_text = '***'
    elif p_val < 1e-2:
        p_text = '**'
    else:
        p_text = '*'
    
    return p_text  # string of asterisks

In [3]:
# chi-square test with multiple hypothesis correction
# following the work from: https://neuhofmo.github.io/chi-square-and-post-hoc-in-python/
def run_chisq_on_combination(df, combinations_tuple):
    """Receives a dataframe and a combinations tuple and returns p-value after performing chisq test."""
    assert len(combinations_tuple) == 2, "Combinations tuple is too long! Should be of size 2."
    new_df = df[(df.index == combinations_tuple[0]) | (df.index == combinations_tuple[1])]
    # print(new_df)
    try:
        chi2, p, dof, ex = chi2_contingency(new_df, correction=True)
    except ValueError:
            print('Expected values is zero in one column for this contingency table: ')
            print(new_df)
            p=100 # setting some extreme values to be removed from the next list
    return p


def chisq_and_posthoc_corrected(df, correction_method='fdr_bh', alpha=0.05):
    """Receives a dataframe and performs chi2 test and then post hoc.
    Prints the p-values and corrected p-values (after FDR correction).
    alpha: optional threshold for rejection (default: 0.05)
    correction_method: method used for mutiple comparisons correction. (default: 'fdr_bh').
    See statsmodels.sandbox.stats.multicomp.multipletests for elaboration."""

    # start by running chi2 test on the matrix
    chi2, p, dof, ex = chi2_contingency(df, correction=True)
    print("Chi2 result of the contingency table: {}, p-value: {}\n".format(chi2, p))
    
    # post-hoc test
    all_combinations = list(combinations(df.index, 2))  # gathering all combinations for post-hoc chi2
    # print(all_combinations)
    print("Post-hoc chi2 tests results:")
    p_vals = [run_chisq_on_combination(df, comb) for comb in all_combinations]  # a list of all p-values
    # the list is in the same order of all_combinations
    # print(all_combinations)
    # print(p_vals)


    # all_combinations = list(filter(lambda item: item > 100, my_list))
    # check if we have any tests, where we could not run chi-square test
    # p-value=100 (as set up in the function above)
    # if item in p_vals == 100:
    # print([s for s in p_vals if s != 100])
    i=100
    if i in p_vals:
        print ('Invalid Chi-Square Test')
        print('Removing that combination for post-hoc testing')
        print(all_combinations)
        print(p_vals)
        del all_combinations[p_vals.index(100)]
        p_vals=[s for s in p_vals if s != 100]
        print('After removing that combination:')
        print(all_combinations)
        print(p_vals)

    
    
    # all_combinations=all_combinations[s in p_vals if s != 100]


    # correction for multiple testing
    reject_list, corrected_p_vals = multipletests(p_vals, method=correction_method, alpha=alpha)[:2]
    for p_val, corr_p_val, reject, comb in zip(p_vals, corrected_p_vals, reject_list, all_combinations):
        print("{}: p_value: {:5f}; corrected: {:5f} ({}) reject: {}".format(comb, p_val, corr_p_val, get_asterisks_for_pval(p_val, alpha), reject))


In [ ]:
# read data
df=pd.read_excel('smartdevice survey individual demographics and answers-deidentified.xlsx',sheet_name='XX') # Code with actual data will be uploaded later


## Statistical Test on Ownership of Smartphone Across Demographic Factors

In [ ]:
# For Gender
# Filter Dataframe to include only Male vs Female as we have very limited representation with participants from other gender types
df_gender=df.loc[(df['Gender']=='Male') | (df['Gender']=='Female')]

smartphone_ownership_gender_crosstab, smartphone_ownership_gender_test_results, smartphone_ownership_gender_expected = rp.crosstab(df_gender["smartphone_ownership_binary_response"], df_gender["Gender"], test= "chi-square", expected_freqs= True, prop= "cell", cramer_correction = True) 


smartphone_ownership_gender_test_results

In [ ]:
# For Race
#Filter Dataframe to include only White, Black, Hispanic, and Asian as we have very limited representation from participants from race and ethnicities
df_race=df.loc[(df['race_ethnicity']=='White or Caucasian') | (df['race_ethnicity']=='Black or African American') | (df['race_ethnicity']=='Asian or Asian American') | (df['race_ethnicity']=='Hispanic')]

smartphone_ownership_race_crosstab, smartphone_ownership_race_test_results, smartphone_ownership_race_expected = rp.crosstab(df_race["smartphone_ownership_binary_response"], df_race["race_ethnicity"], test= "chi-square", expected_freqs= True, prop= "cell", cramer_correction = True) 


smartphone_ownership_race_test_results

In [ ]:
# For Age
wearable_ownership_age_crosstab, wearable_ownership_age_test_results, wearable_ownership_age_expected = rp.crosstab(df["wearable_ownership_binary_response"], df["generation_age_group"], test= "chi-square", expected_freqs= True, prop= "cell", cramer_correction = True) 

wearable_ownership_age_test_results


In [ ]:
# Run Chi-Square Test with Multiple Hypotheses Correction
chisq_and_posthoc_corrected(pd.crosstab(df_age['generation_age_group'], df_age['wearable_ownership_binary_response']))

In [ ]:
# For Highest Level of Education 
# Filter Dataframe to exclude participants with less than high school education as we have very limited representation with people with less than high school educaiton
df_education=df.loc[(df['Highest level of education']!='Less than high school') ]


smartphone_ownership_education_crosstab, smartphone_ownership_education_test_results, smartphone_ownership_education_expected = rp.crosstab(df_education["smartphone_ownership_binary_response"], df_education["Highest level of education"], test= "chi-square", expected_freqs= True, prop= "cell", cramer_correction = True) 


smartphone_ownership_education_test_results

In [ ]:
# Run Chi-Square Test with Multiple Hypotheses Correction
chisq_and_posthoc_corrected(pd.crosstab(df_education["Highest level of education"], df_education["smartphone_ownership_binary_response"]))

In [ ]:
# For employment status
smartphone_ownership_employment_crosstab, smartphone_ownership_employment_test_results, smartphone_ownership_employment_expected = rp.crosstab(df["smartphone_ownership_binary_response"], df["Employment Status"], test= "chi-square", expected_freqs= True, prop= "cell", cramer_correction = True) 

smartphone_ownership_employment_test_results

In [ ]:
# Run Chi-Square Test with Multiple Hypotheses Correction
chisq_and_posthoc_corrected(pd.crosstab(df['Employment Status'], df['smartphone_ownership_binary_response']))

## Figures

## Figure: Motivation for ownership of wearables

In [9]:
#Load data
wearable_ownership_motivation_df=pd.read_excel('../duke_health_listens_smart_device_ownership_data.xlsx',sheet_name='Q7_wearable_device_usage_reason',index_col=0)
#sorting DF by Main Reason
wearable_ownership_motivation_df.sort_values(by=['Main reason'],inplace=True,ascending=True)

# Generate Figure:
list_of_colors=['tab:blue', 'tab:orange', 'tab:gray','tab:olive']
list_of_colors=['#005587', '#0577B1','#ADD8E6','#B5B5B5']
fig, ax = plt.subplots(1,1,figsize=(6,4))
plt.rcParams.update({'font.size': 10, 'font.family':'sans-serif','font.sans-serif': 'Arial' })
plt.rcParams['axes.grid'] = True
wearable_ownership_motivation_df.plot(kind='barh', stacked=True, color=list_of_colors,ax=ax)

plt.legend(ncol=2,bbox_to_anchor=(-0.3, -0.4),loc='lower left')
ax.set_xlim(0, 100)
ax.set_ylabel('')
ax.set_xlabel('Motivation Priority (%)')

ax.set_xticks([0, 25, 50, 75, 100])
fig.tight_layout()
figurename_to_save='Figures/wearable_ownership_motivation.jpg'
plt.savefig(figurename_to_save,format='jpg', dpi = 600)
plt.show()


## Figure: Willingess to share data from smartdevices for research purposes

In [ ]:
#Load data
data_sharing_willingness_df=pd.read_excel('../duke_health_listens_smart_device_ownership_data.xlsx',sheet_name='Q_types_of_data_comfortable_sha',index_col=0)
data_sharing_willingness_df.sort_values(by=['Yes'],inplace=True,ascending=True)
data_sharing_willingness_df=data_sharing_willingness_df.iloc[:,[0,2,1]] 

# Generate Figure:
fig, ax = plt.subplots(1,1,figsize=(5,3))
list_of_colors=['#005587', '#ADD8E6','#B5B5B5']

plt.rcParams.update({'font.size': 10, 'font.family':'sans-serif','font.sans-serif': 'Arial' })

data_sharing_willingness_df.plot(kind='barh', stacked=True, color=list_of_colors,ax=ax,zorder=3)

mylabels=['Willing', 'Maybe Willing', 'Not Willing']
plt.legend(labels=mylabels, ncol=3,bbox_to_anchor=(-0.6, -0.5),loc='lower left')
ax.set_xlim(0, 100)
ax.set_ylabel('Types of Data')
ax.set_xlabel('Data Sharing Acceptability (%)')
ax.grid(True, linestyle='--',axis='x',zorder=0)
ax.set_xticks([0, 25, 50, 75, 100])
fig.tight_layout()
figurename_to_save='Figures/data_sharing_willingness.jpg'
plt.savefig(figurename_to_save,format='jpg', dpi = 600)
plt.show()